In [1]:
import pandas as pd       
data1 = pd.read_csv("yelp_reviews_1_Chinese.csv")
data0 = pd.read_csv("yelp_reviews_0_Chinese.csv")
i_0, j_0 = data0.shape
i_1, j_1 = data1.shape

train =  [data0.head(int(i_0*0.8)),data1.head(int(i_1*0.8))]
test =  [data0.tail(i_0-int(i_0*0.8)),data1.tail(i_1 - (int(i_1*0.8)))]
train = pd.concat(train)
test = pd.concat(test)
train = train.reset_index() 
test = test.reset_index()

In [2]:
from nltk.tag import pos_tag
import nltk
from bs4 import BeautifulSoup    
import re
from nltk.corpus import stopwords # Import the stop word list
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw restaurant review), and 
    # the output is a single string (a preprocessed restaurant review)
    #
    
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    # save only noun
    meaningful_words = pos_tag(meaningful_words)
    meaningful_words = [word for word,pos in meaningful_words if pos == 'NNS']
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [3]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["Review"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the restaurant review list 
for i in range( 0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_words( train["Review"][i] ) )

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [4]:
print ("Cleaning and parsing the training restaurant reviews...\n")
clean_train_reviews = []
for i in range( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print ("Review %d of %d\n" % ( i+1, num_reviews ) )                                                                   
    clean_train_reviews.append( review_to_words( train["Review"][i] ))

Cleaning and parsing the training restaurant reviews...

Review 1000 of 1600



In [5]:
print ("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [6]:
 train_data_features.shape

(1600, 5000)

In [7]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print (vocab)

[u'abandon', u'abilities', u'accents', u'accepts', u'accessories', u'accidents', u'accolades', u'accommodations', u'accompanies', u'accompaniments', u'accomplishments', u'accounts', u'accoutrements', u'accusations', u'aces', u'aches', u'achieve', u'acknowledges', u'acoustics', u'acquaintances', u'acres', u'actions', u'activities', u'actors', u'acts', u'acupuncturists', u'adams', u'add', u'addicts', u'additions', u'additives', u'addresses', u'adjustments', u'admissions', u'admit', u'adores', u'ads', u'adults', u'advantages', u'adventures', u'advertisements', u'advertises', u'adverts', u'aerobics', u'aesthetics', u'affairs', u'afternoons', u'afterwards', u'afterwords', u'agents', u'ages', u'agnolotti', u'ah', u'ahi', u'aids', u'aioli', u'airlines', u'aisles', u'aka', u'al', u'alas', u'albeit', u'albertsons', u'alcoholics', u'ales', u'alfajores', u'alfredo', u'allergies', u'allows', u'almonds', u'alors', u'als', u'alterations', u'alternatives', u'amateurs', u'amazeballs', u'amen', u'amend

In [8]:
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print (count, tag)

(3, u'abandon')
(4, u'abilities')
(31, u'accents')
(11, u'accepts')
(38, u'accessories')
(9, u'accidents')
(10, u'accolades')
(20, u'accommodations')
(7, u'accompanies')
(22, u'accompaniments')
(4, u'accomplishments')
(18, u'accounts')
(8, u'accoutrements')
(5, u'accusations')
(7, u'aces')
(10, u'aches')
(3, u'achieve')
(4, u'acknowledges')
(23, u'acoustics')
(5, u'acquaintances')
(3, u'acres')
(8, u'actions')
(32, u'activities')
(11, u'actors')
(34, u'acts')
(3, u'acupuncturists')
(12, u'adams')
(12, u'add')
(4, u'addicts')
(24, u'additions')
(5, u'additives')
(4, u'addresses')
(15, u'adjustments')
(3, u'admissions')
(7, u'admit')
(3, u'adores')
(18, u'ads')
(166, u'adults')
(3, u'advantages')
(18, u'adventures')
(8, u'advertisements')
(7, u'advertises')
(3, u'adverts')
(4, u'aerobics')
(16, u'aesthetics')
(4, u'affairs')
(28, u'afternoons')
(177, u'afterwards')
(4, u'afterwords')
(5, u'agents')
(66, u'ages')
(3, u'agnolotti')
(3, u'ah')
(4, u'ahi')
(6, u'aids')
(5, u'aioli')
(4, u'ai

In [9]:
print ("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["Polarity"] )

Training the random forest...


In [10]:
feasure = np.array(vocab)
importances = forest.feature_importances_
index = importances.argsort()[-50:]
index

array([4516, 4534, 4048, 4446, 1394, 1198, 2070,  565, 3837,  643, 3358,
       4071, 3199, 3711, 4365, 3625, 2694,  990, 3135, 4149, 3712, 4192,
       3317,  764, 4961, 4486, 1161, 3753, 3737, 3482,  548, 3413,  519,
       3252, 2583, 2350, 1445, 2981, 3243,  770, 1683, 3601, 1367, 3296,
       4722, 4918, 4718, 3670, 2884, 1257])

In [11]:
feasure[index]

array([u'tomatoes', u'toppings', u'soups', u'things', u'eggrolls', u'des',
       u'hours', u'burgers', u'servers', u'carrots', u'prices',
       u'specials', u'pieces', u'salads', u'tastes', u'reviews',
       u'minutes', u'containers', u'people', u'stars', u'sales', u'stores',
       u'potstickers', u'china', u'years', u'times', u'days', u'sauces',
       u'sandwiches', u'rangoons', u'buffets', u'puffs', u'broccoli',
       u'plates', u'meals', u'leftovers', u'entrees', u'orders', u'places',
       u'chinese', u'fries', u'restaurants', u'dumplings', u'portions',
       u'veggies', u'wontons', u'vegetables', u'rolls', u'noodles',
       u'dishes'], 
      dtype='<U17')

In [12]:
# Read the test data


# Verify that there are 25,000 rows and 2 columns
print (test.shape)

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["Review"])
clean_test_reviews = [] 

print ("Cleaning and parsing the test set restaurant reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print ("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["Review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["Business_id"], "Polarity":result} )
output
# Use pandas to write the comma-separated output file
#output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

(402, 4)
Cleaning and parsing the test set restaurant reviews...



,Polarity,id
0,0,b'wLlxavQWA1-d9Pcwp9nbLg'
1,0,b'1_lU0-eSWJCRvNGk78Zh9Q'
2,0,b'PSlOtPIo28BvWOzdLmRRRQ'
3,0,b'v3r3IFCiWYrae-n3vyccSA'
4,0,b'QjMZ3Gt4c-vS2uVUWQrVbw'
5,0,b'YRDKsBFYP7G0E8BLJKIkvA'
6,0,b'3OViesv3-RkPDyvdxW16sg'
7,0,b'_yZbIA76UR0cIgoq0jM5zQ'
8,0,b'P1fJb2WQ1mXoiudj8UE44w'
9,0,b'3UL1To_gn_eOY5iFH5oBxA'
